### test the model on given english test data and then compare with test gold data

In [1]:
import pandas as pd
import polars as pl
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import string
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import pipeline
# import pandas as pd
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
# import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report

from sklearn.decomposition import PCA
import joblib

import spacy
from collections import Counter
import numpy as np

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

# 1. load the data

In [2]:
# change to test_english
df = pd.read_csv('data/CT24_checkworthy_english/CT24_checkworthy_english/tset_english/CT24_checkworthy_english_test.tsv', delimiter = '\t')

In [5]:
#df.drop('raw_text_length', axis = 1, inplace = True)
df.head()

,Sentence_id,Text
0,35988,They said they were just going to get inspecto...
1,35991,"And from that point on, I've voted to -- I mov..."
2,36029,I sit on the Senate Armed Services Committee.
3,36043,We need to depend on all of our tools -- diplo...
4,36063,And that is -- and I don't know if my colleagu...


In [4]:
# new label from text to 0/1
# df['label'] = df['class_label'].apply(lambda x: 1 if x == 'Yes' else 0)

In [5]:
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\looka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\looka\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 2. generates tokens

In [6]:
def clean_text(text):
    # remove punctuation
    text = ''.join([char for char in text if char not in string.punctuation]) # 
    # tokenize
    tokens = word_tokenize(text)
    # remove stop words
    token = [word for word in tokens if word.lower() not in stop_words]
    return token

In [7]:
# creating new column name Token
df['Tokens'] = df['Text'].apply(clean_text)

# 3. row text length

In [8]:
def verious_text_length(text):
    text_length = len(text)
    return text_length

In [9]:
df['text_length'] = df['Text'].apply(verious_text_length)

In [10]:
def cleaned_text(text):
    # lowercase the data
    lower_text = text.lower()
    # removing the punctuations
    remove_punct = ''.join([char for char in lower_text if char not in string.punctuation])
    # remove the stop words
    words = word_tokenize(remove_punct)
    remove_stop_words = [word for word in words if word not in stop_words]
    # removing the numbers
    # removing the extra space
    # replace the repetations of punctuations
    # removing emojis
    # removing emoticons
    # removing contractions
    
    return ' '.join(remove_stop_words)

# 4. clean text and cleaned text length

In [12]:
df['cleaned_text'] = df['Text'].apply(cleaned_text)

In [13]:
df['cleaned_text_length'] = df['cleaned_text'].apply(verious_text_length)

In [14]:
# nltk.download('vader_lexicon')

In [15]:
# sia = SentimentIntensityAnalyzer()

In [16]:
# def get_vader_sentiment_score(text):
#     return sia.polarity_scores(text)

In [17]:
# df_new['sentiment_scores'] = df_new['Text'].apply(get_vader_sentiment_score) 
# # extract individual sentiment into seperate columns
# df_new = pd.concat([df_new.drop(['sentiment_scores'], axis = 1), 
#                    df_new['sentiment_scores'].apply(pd.Series)], axis = 1).rename(columns= {'neg' : 'vader_neg', 'neu': 'vader_neu','pos' : 'vader_pos','compound' : 'vader_compound'})


In [19]:
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

In [20]:
def get_sentiment_features(text):
    # Tokenize the input text
    tokens = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    
    # Perform inference
    outputs = model(**tokens)
    logits = outputs.logits
    
    # Apply softmax to calculate probabilities
    probs = torch.nn.functional.softmax(logits, dim=-1)
    negative, neutral, positive = probs[0].tolist()
    
    # Define thresholds for "mixed" classification (adjust as needed)
    if max([positive, negative, neutral]) == positive and positive > 0.5:
        sentiment = "Positive"
    elif max([positive, negative, neutral]) == negative and negative > 0.5:
        sentiment = "Negative"
    else:
        sentiment = "Mixed"
    
    return {
        "positive_score": positive,
        "negative_score": negative,
        "neutral_score": neutral,
        "sentiment_class": sentiment
    }

# 5. roberta sentiment generate, then different sentiment put in seperate coulumn

In [22]:
df['roberta_sentiment'] = df['Text'].apply(get_sentiment_features) 

In [15]:
df["roberta_sent_pos"] = df["roberta_sentiment"].apply(lambda x: x["positive_score"])
df["roberta_sent_neg"] = df["roberta_sentiment"].apply(lambda x: x["negative_score"])
df["roberta_sent_mixed"] = df["roberta_sentiment"].apply(lambda x: x["neutral_score"])
df["roberta_sentiment_class"] = df["roberta_sentiment"].apply(lambda x: x["sentiment_class"])

In [16]:
# df_new["is_positive"] = df_new["sentiment_class"].apply(lambda x: 1 if x == "Positive" else 0)
# df_new["is_negative"] = df_new["sentiment_class"].apply(lambda x: 1 if x == "Negative" else 0)
# df_new["is_mixed"] = df_new["sentiment_class"].apply(lambda x: 1 if x == "Mixed" else 0)

In [17]:
def count_tokens(tokens):
    return len(tokens)

# 6. token length

In [18]:
df['token_length'] = df['Tokens'].apply(count_tokens)

In [19]:
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", grouped_entities=True)

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
C:\Users\looka\OneDrive\Documents\Thesis\master_thesis\lib\site-packages\transformers\pipelines\token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


In [ ]:
# def entity_counts(text):
#     entities = ner_pipeline(text)
    
#     # Initialize counts for each entity type
#     name_count = 0
#     organization_count = 0
#     date_count = 0
    
#     # Count entities based on their type
#     for entity in entities:
#         if entity["entity_group"] == "PER":  # Person
#             name_count += 1
#         elif entity["entity_group"] == "ORG":  # Organization
#             organization_count += 1
#         elif entity["entity_group"] == "DATE":  # Date
#             date_count += 1
    
#     # Return the counts as a pandas Series
#     return pd.Series([name_count, organization_count, date_count])



In [ ]:
# df_new[[a'name_count', 'organization_count', 'date_count']] = df_new['Text'].apply(entity_counts)

In [20]:
nlp = spacy.load("en_core_web_sm")

In [21]:
def count_entities_to_columns(text):
    

    # # Process each row in the DataFrame
    # for text in df[text_column]:
    #     # Default counts for each row
    names = 0
    orgs = 0
    dates = 0

        # Process text using spaCy
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            names += 1
        elif ent.label_ == "ORG":
            orgs += 1
        elif ent.label_ == "DATE":
            dates += 1

    return {
        "names": names,
        "organizations": orgs,
        "dates": dates
    }

# 7. count name organization and date, then out in the seperate columns

In [22]:
df['counts'] = df['Text'].apply(count_entities_to_columns)

In [23]:
df["names"] = df["counts"].apply(lambda x: x["names"])
df["organizations"] = df["counts"].apply(lambda x: x["organizations"])
df["dates"] = df["counts"].apply(lambda x: x["dates"])
# df_new["sentiment_class"] = df_new["roberta_sentiment"].apply(lambda x: x["sentiment_class"])

In [24]:
def count_punctuation(text):
    # Define a regular expression pattern to match punctuation characters
    punctuation_pattern = r'[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]'
    # Find all punctuation marks in the text
    punctuation_count = len(re.findall(punctuation_pattern, text))
    return punctuation_count

# 8. count punctuations

In [25]:
df['punctuation_count'] = df['Text'].apply(count_punctuation)

In [26]:
# df_new.to_csv('07_01_data.csv', index = False)

In [27]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [28]:
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True, padding="max_length")
    output = model(**inputs)
    embedding = output.last_hidden_state[:, 0, :]
    return embedding.detach().numpy()

# 9. generate embeddings

In [29]:
df['embedding'] = df['Text'].apply(get_embeddings)

In [30]:
# convert to numpy array
def convert_to_numpy_array(embd_str):
    #print(f"string: {embd_str}")
    #print(type(embd_str))
    # embd_str_clean = embd_str.replace("\n", " ").replace(" ", " ")
    #print(type(embd_str_clean))
    #print(f"string cleansing: {embd_str_clean}")
    # embd_str_clean = embd_str.strip('[]')
    #print(type(embd_str_clean))
    #print(f"removing the outer brackets {embd_str_clean}")
    #embd_list = embd_str.split()
    #print(f"Split the string into individual values {embd_list}")
    embed_array = np.array(embd_str, dtype = float)
    #print(f"list to array {embed_array}")
    return embed_array

# 10. renaming the column name , similler to the column name, when the model was trained

In [31]:
df.rename(columns = {'token_length': 'count_tokens'}, inplace = True)

In [32]:
df.columns

Index(['Sentence_id', 'Text', 'Tokens', 'text_length', 'cleaned_text',
       'cleaned_text_length', 'roberta_sentiment', 'roberta_sent_pos',
       'roberta_sent_neg', 'roberta_sent_mixed', 'roberta_sentiment_class',
       'count_tokens', 'counts', 'names', 'organizations', 'dates',
       'punctuation_count', 'embedding'],
      dtype='object')

# 11. convert the embedding into numoy array for model input

In [33]:
df['converted_embedding'] = df['embedding'].apply(convert_to_numpy_array)

In [34]:
df = df.astype({'text_length': 'int32', 'roberta_sent_neg': 'float32', 'roberta_sent_mixed': 'float32', 'roberta_sent_pos': 'float32',
               'names': 'int32', 'organizations': 'int32', 'dates': 'int32', 'count_tokens': 'int32', 'cleaned_text_length': 'int32', 'punctuation_count': 'int32'})

In [35]:
print(df['converted_embedding'].iloc[0].shape)

(1, 768)


# 12. flatten the embedding

In [36]:
df['flattened_embedding'] = df['converted_embedding'].apply(lambda x: np.squeeze(x))

In [117]:
# df['flattened_embedding'].iloc[0]

In [ ]:
# save in csv file

In [45]:
df.to_csv('english_test_08_01.csv', index = False)

# 13. PCA for reducing dimensionality of embedding

In [37]:
pca = PCA(n_components = 50)
reduced_embeddings = pca.fit_transform(df['flattened_embedding'].to_list())

In [38]:
reduced_embeddings = np.array(reduced_embeddings, dtype = np.float32)

In [39]:
reduced_embeddings.shape

(341, 50)

In [40]:
# converted_embedding = np.vstack(df['converted_embedding'])

In [41]:
additional_features = df[['text_length', 'roberta_sent_neg', 'roberta_sent_mixed',
                            'roberta_sent_pos', 'names', 'organizations', 'dates',
                            'count_tokens', 'cleaned_text_length', 'punctuation_count']].to_numpy()

In [46]:
X = np.hstack([reduced_embeddings, additional_features]) # stack horizontally to add additional features with corresponding embedding vectors
# y = df['labels']

In [47]:
model_path_lr = 'models/logistic_reggression.pkl'
model_lr = joblib.load(model_path_lr)
predictions_lr = model_lr.predict(X)
predictions_lr

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [48]:
model_path_rfc = 'models/rfc.pkl'
model_rfc = joblib.load(model_path_rfc)
predictions_rfc = model_rfc.predict(X)
predictions_rfc

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [49]:
test_data_lr = pd.DataFrame({
    'id': df['Sentence_id'],
    'predictions': predictions_lr
})

In [50]:
test_data_lr['predictions'] = test_data_lr['predictions'].apply([lambda x: 'No' if x == 0 else 'Yes'])

In [62]:
print(test_data_lr[test_data_lr['predictions'] == 'Yes'].count())
print(test_data_lr[test_data_lr['predictions'] == 'No'].count())

id             18
predictions    18
dtype: int64
id             323
predictions    323
dtype: int64


In [51]:
test_data_rfc = pd.DataFrame({
    'id': df['Sentence_id'],
    'predictions': predictions_rfc
})

In [52]:
test_data_rfc['predictions'] = test_data_rfc['predictions'].apply([lambda x: 'No' if x == 0 else 'Yes'])

In [63]:
print(test_data_rfc[test_data_rfc['predictions'] == 'Yes'].count())
print(test_data_rfc[test_data_rfc['predictions'] == 'No'].count())

id             18
predictions    18
dtype: int64
id             323
predictions    323
dtype: int64


In [53]:
gold_df = pd.read_csv('data/CT24_checkworthy_english/CT24_checkworthy_english/test_english_gold/CT24_checkworthy_english_test_gold.tsv',delimiter = '\t')

In [54]:
gold_df.head()

,Sentence_id,Text,class_label
0,35988,They said they were just going to get inspecto...,Yes
1,35991,"And from that point on, I've voted to -- I mov...",Yes
2,36029,I sit on the Senate Armed Services Committee.,No
3,36043,We need to depend on all of our tools -- diplo...,No
4,36063,And that is -- and I don't know if my colleagu...,No


In [58]:
## for LR
df_merged_lr = pd.merge(gold_df, test_data_lr, left_on='Sentence_id', right_on='id', how='inner')

# Correct the typo: 'gold_lables' to 'gold_labels'
gold_labels = df_merged_lr['class_label'].tolist()
pred_labels = df_merged_lr['predictions'].tolist()

# Now use 'gold_labels' consistently
acc = accuracy_score(gold_labels, pred_labels)
precision = precision_score(gold_labels, pred_labels, pos_label='Yes', average='binary')
recall = recall_score(gold_labels, pred_labels, pos_label='Yes', average='binary')
f1 = f1_score(gold_labels, pred_labels, pos_label='Yes', average='binary')

# Print the metrics
print(f"Accuracy: {acc}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.7360703812316716
Precision: 0.4444444444444444
Recall: 0.09090909090909091
F1 Score: 0.1509433962264151


In [59]:
# for rfc
df_merged_rfc = pd.merge(gold_df, test_data_rfc, left_on='Sentence_id', right_on='id', how='inner')

# Correct the typo: 'gold_lables' to 'gold_labels'
gold_labels = df_merged_rfc['class_label'].tolist()
pred_labels = df_merged_rfc['predictions'].tolist()

# Now use 'gold_labels' consistently
acc = accuracy_score(gold_labels, pred_labels)
precision = precision_score(gold_labels, pred_labels, pos_label='Yes', average='binary')
recall = recall_score(gold_labels, pred_labels, pos_label='Yes', average='binary')
f1 = f1_score(gold_labels, pred_labels, pos_label='Yes', average='binary')

# Print the metrics
print(f"Accuracy: {acc}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.7360703812316716
Precision: 0.4444444444444444
Recall: 0.09090909090909091
F1 Score: 0.1509433962264151


## 